In [1]:
# Import necessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Always make it pretty.
plt.style.use('ggplot')
%matplotlib inline
sns.set_style(style="whitegrid")

In [4]:
# Load the data
business_json_path = 'data\yelp_dataset\yelp_dataset~\yelp_academic_dataset_business.json'
business_df = pd.read_json(business_json_path, lines=True)

In [6]:
review_json_path = 'data\yelp_dataset\yelp_dataset~\yelp_academic_dataset_review.json'
review_df = pd.read_json(review_json_path, lines=True)

MemoryError: 